## CSC 820 NLT
## HW11
* > Khalid Mehtab Khan
* > 923673423

## Program_0    

Text Mining : TF-IDF and Cosine Similarity from Scratch

Table of Contents:

1. Term Frequency (TF)
2. Inverse Document Frequency (IDF)
3. TF * IDF
4. Vector Space Models and Representation – Cosine Similarity

*** Any feedback or feature requests are welcome!***


Let us imagine that you are doing a search on below documents with the following query: **life learning**

* **Document 1** : I want to start learning to charge something in life.
* **Document 2** : learning something about me no one else knows
* **Document 3** : Never stop learning

The query is a free text query. It means a query in which the terms of the query are typed freeform into the search interface, without any connecting search operators.

Let us go over each step in detail to see how it all works.

# 1.Term Frequency(TF)
Term Frequency also known as TF measures the number of times a term (word) occurs in a document. Given below is the code and the terms and their frequency on each of the document.


In [1]:
# Imprting required libraries
# math for mathematical operations
# numpy and pandas for data manipulation

In [2]:
import math
import pandas as pd
import numpy as np

In [3]:
# Defining a Toy dataset
# 3 documents
# 1 query

In [4]:
#documents
doc1 = "I want to start learning to charge something in life"
doc2 = "reading something about life no one else knows"
doc3 = "Never stop learning"
#query string
query = "life learning"

In [5]:
# documents are stored in a list

In [6]:
documents = [doc1, doc2, doc3]
documents

['I want to start learning to charge something in life',
 'reading something about life no one else knows',
 'Never stop learning']

**NOTE :** Text Preprocessing Steps are ignored as the objective of this kernel is to explain and develop TF-IDF and cosine similarity from scratch

In [7]:
# compute-tf function

# This function computes the frequency of each word in the document
# For each document in the list of documents,
# we split the document into words and count the frequency of each word
# We store the frequency of each word in a temperory dictionary
# Word frequencies for each document is printed at the end of each iteration

In [8]:
#term -frequenvy :word occurences in a document
def compute_tf(docs_list):
    for doc in docs_list:
        doc1_lst = doc.split(" ")
        wordDict_1= dict.fromkeys(set(doc1_lst), 0)

        for token in doc1_lst:
            wordDict_1[token] +=  1
        df = pd.DataFrame([wordDict_1])
        idx = 0
        new_col = ["Term Frequency"]    
        df.insert(loc=idx, column='Document', value=new_col)
        print(df)
        
compute_tf(documents)

         Document  to  want  start  I  in  something  learning  life  charge
0  Term Frequency   2     1      1  1   1          1         1     1       1
         Document  knows  no  one  about  something  reading  else  life
0  Term Frequency      1   1    1      1          1        1     1     1
         Document  stop  Never  learning
0  Term Frequency     1      1         1


In [9]:
# We can see documents are like a data frame now

# Sentences are toekinzed into words
# for each token we have a frequency count in that document

* In reality each document will be of different size. On a large document the frequency of the terms will be much higher than the smaller ones. Hence we need to normalize the document based on its size. 
* A simple trick is to divide the term frequency by the total number of terms. 
* For example in Document 1 the term game occurs two times. The total number of terms in the document is 10. Hence the normalized term frequency is 2 / 10 = 0.2. 


Given below are the normalized term frequency for all the documents.

In [10]:
# compute_normalizedtf function

# This function calculates the normalized term frequency for each word in a document.
# It first breaks down each document into words and converts them to lowercase.
# Each word's frequency in the document is divided by the total number of words to get its normalized frequency.
# This frequency is stored in a dictionary. tf_doc

# It also displays a DataFrame for each document, showing these frequencies under the column "Normalized TF".
# The function returns a list of these dictionaries, one for each document.

In [12]:
#Normalized Term Frequency
def termFrequency(term, document):
    normalizeDocument = document.lower().split()
    return normalizeDocument.count(term.lower()) / float(len(normalizeDocument))

def compute_normalizedtf(documents):
    tf_doc = []
    for txt in documents:
        sentence = txt.split()
        norm_tf= dict.fromkeys(set(sentence), 0)
        for word in sentence:
            norm_tf[word] = termFrequency(word, txt)
        tf_doc.append(norm_tf)
        df = pd.DataFrame([norm_tf])
        idx = 0
        new_col = ["Normalized TF"]    
        df.insert(loc=idx, column='Document', value=new_col)
        print(df)
    return tf_doc

tf_doc = compute_normalizedtf(documents)


        Document   to  want  start    I   in  something  learning  life  \
0  Normalized TF  0.2   0.1    0.1  0.1  0.1        0.1       0.1   0.1   

   charge  
0     0.1  
        Document  knows     no    one  about  something  reading   else   life
0  Normalized TF  0.125  0.125  0.125  0.125      0.125    0.125  0.125  0.125
        Document      stop     Never  learning
0  Normalized TF  0.333333  0.333333  0.333333


- This is the normalized word frequency of every word in each document.

In [14]:
tf_doc

[{'to': 0.2,
  'want': 0.1,
  'start': 0.1,
  'I': 0.1,
  'in': 0.1,
  'something': 0.1,
  'learning': 0.1,
  'life': 0.1,
  'charge': 0.1},
 {'knows': 0.125,
  'no': 0.125,
  'one': 0.125,
  'about': 0.125,
  'something': 0.125,
  'reading': 0.125,
  'else': 0.125,
  'life': 0.125},
 {'stop': 0.3333333333333333,
  'Never': 0.3333333333333333,
  'learning': 0.3333333333333333}]

In [16]:
# For the first document, 
# words like "life," "I," "start," and others appear once in the document, so they have the same normalized frequency of 0.1. AS there are 10 words.
# In the third document, the words "Never," "stop," and "learning" all have an equal frequency of 0.333333, 
# since there are only three words in this document.

# 2. Inverse Document Frequency (IDF)

* The main purpose of doing a search is to find out relevant documents matching the query. 
* In Term Frequecy all terms are considered equally important. In fact certain terms that occur too frequently have little power in determining the relevance. 
* We need a way to weigh down the effects of too frequently occurring terms. Also the terms that occur less in the document can be more relevant. 
* We need a way to weigh up the effects of less frequently occurring terms. Logarithms helps us to solve this problem.Logarithms helps us to solve this problem.


Let us compute IDF for the term start

IDF(start) = 1 + loge(Total Number Of Documents / Number Of Documents with term start in it)

There are 3 documents in all = Document1, Document2, Document3
The term start appears in Document1

 IDF(start) = 1 + loge(3 / 1)
            = 1 + 1.098726209
            = 2.098726209

In [17]:
# compute_idf function

# This function figures out how unique a word is across all documents.
# It checks how many documents contain each word.
# The more documents a word is in, the less unique it is.
# We do some math to find a score (IDF) that gets bigger the rarer the word is.
# If a word isn't in any documents, we just give it a score of 1.
# At the end, it gives us a list with all the words and their uniqueness scores.

In [18]:
def inverseDocumentFrequency(term, allDocuments):
    numDocumentsWithThisTerm = 0
    for doc in range (0, len(allDocuments)):
        if term.lower() in allDocuments[doc].lower().split():
            numDocumentsWithThisTerm = numDocumentsWithThisTerm + 1
 
    if numDocumentsWithThisTerm > 0:
        return 1.0 + math.log(float(len(allDocuments)) / numDocumentsWithThisTerm)
    else:
        return 1.0
    
def compute_idf(documents):
    idf_dict = {}
    for doc in documents:
        sentence = doc.split()
        for word in sentence:
            idf_dict[word] = inverseDocumentFrequency(word, documents)
    return idf_dict
idf_dict = compute_idf(documents)

compute_idf(documents)

{'I': 2.09861228866811,
 'want': 2.09861228866811,
 'to': 2.09861228866811,
 'start': 2.09861228866811,
 'learning': 1.4054651081081644,
 'charge': 2.09861228866811,
 'something': 1.4054651081081644,
 'in': 2.09861228866811,
 'life': 1.4054651081081644,
 'reading': 2.09861228866811,
 'about': 2.09861228866811,
 'no': 2.09861228866811,
 'one': 2.09861228866811,
 'else': 2.09861228866811,
 'knows': 2.09861228866811,
 'Never': 2.09861228866811,
 'stop': 2.09861228866811}

In [19]:
# The numbers next to each word are the IDF scores, which tell us how common or rare a word is across all the documents you've used. 
# For example, the word "learning" has a score of about 1.4, 
# meaning it's less common across your documents than the word "I", which has a score of roughly 2.1. 
# The higher the score, the rarer the word. 
# These scores help in understanding which words might carry more weight in terms of uniqueness in your set of documents.

# 3.TF * IDF

Remember we are trying to find out relevant documents for the query: **life learning**

* For each term in the query multiply its normalized term frequency with its IDF on each document. 
* In Document1 for the term life the normalized term frequency is 0.1 and its IDF is 1.405465108. 
* Multiplying them together we get 0.140550715 (0.1 * 1.405465108). 
* 
Given below is TF * IDF calculations for life and learning in all the documents.

In [20]:
# compute_tfidf_with_alldocs function

# This function calculates the importance of each search term within each document.
# For each document, it takes the term frequency (TF) of the search term, 
# which is how often the term appears in that particular document.
# Then, it looks at how common or rare the term is across all documents, which is the inverse document frequency (IDF).
# By multiplying the TF by the IDF, we get the TF-IDF score, which tells us how relevant each term is in every document.
# It does this for each term in the search query and for each document.
# Finally, it provides us with a score for the search query for each document, 
# helping to identify the most relevant documents for the given search terms.

In [21]:
# tf-idf score across all docs for the query string("life learning")
def compute_tfidf_with_alldocs(documents , query):
    tf_idf = []
    index = 0
    query_tokens = query.split()
    df = pd.DataFrame(columns=['doc'] + query_tokens)
    for doc in documents:
        df['doc'] = np.arange(0 , len(documents))
        doc_num = tf_doc[index]
        sentence = doc.split()
        for word in sentence:
            for text in query_tokens:
                if(text == word):
                    idx = sentence.index(word)
                    tf_idf_score = doc_num[word] * idf_dict[word]
                    tf_idf.append(tf_idf_score)
                    df.iloc[index, df.columns.get_loc(word)] = tf_idf_score
        index += 1
    df.fillna(0 , axis=1, inplace=True)
    return tf_idf , df
            
tf_idf , df = compute_tfidf_with_alldocs(documents , query)
print(df)

   doc      life  learning
0  0.0  0.140547  0.140547
1  1.0  0.175683  0.000000
2  2.0  0.000000  0.468488


In [22]:
# Document 2 doesn't mention "life", hence the score of 0.0000, 
# but has a high relevance for "learning", 
# with the score being 0.4685,

# 4.Vector Space Models and Representation  – Cosine Similarity

The set of documents in a collection then is viewed as a set of vectors in a vector space. Each term will have its own axis. Using the formula given below we can find out the similarity between any two documents.

* > Cosine Similarity (d1, d2) =  Dot product(d1, d2) / ||d1|| * ||d2||
* > Dot product (d1,d2) = d1[0] * d2[0] + d1[1] * d2[1] * … * d1[n] * d2[n]
* > ||d1|| = square root(d1[0]2 + d1[1]2 + ... + d1[n]2)
* > ||d2|| = square root(d2[0]2 + d2[1]2 + ... + d2[n]2)


* Vectors deals only with numbers. In this example we are dealing with text documents. This was the reason why we used TF and IDF to convert text into numbers so that it can be represented by a vecto


The query entered by the user can also be represented as a vector. We will calculate the TF*IDF for the query

In [23]:
# compute_query_tf function

# This function takes a search query and finds out how often each word appears in it.
# It splits the query into words and uses the termFrequency function to calculate each word's normalized term frequency.
# This tells us the proportion of each word in the query, considering the length of the query.
# The result is a dictionary where each word from the query is a key, and its normalized term frequency is the value.

In [24]:
#Normalized TF for the query string("life learning")
def compute_query_tf(query):
    query_norm_tf = {}
    tokens = query.split()
    for word in tokens:
        query_norm_tf[word] = termFrequency(word , query)
    return query_norm_tf
query_norm_tf = compute_query_tf(query)
print(query_norm_tf)

{'life': 0.5, 'learning': 0.5}


In [25]:
# compute_query_idf function

# This function works out how common or rare each word in your search query is across all the documents.
# It splits your query into separate words and then calculates the IDF for each one.
# It uses the inverseDocumentFrequency function we talked about before.
# The function creates a dictionary with words from the query as keys and their IDF scores as values.

In [26]:
#idf score for the query string("life learning")
def compute_query_idf(query):
    idf_dict_qry = {}
    sentence = query.split()
    
    for word in sentence:
        idf_dict_qry[word] = inverseDocumentFrequency(word ,documents)
    return idf_dict_qry
idf_dict_qry = compute_query_idf(query)
print(idf_dict_qry)

{'life': 1.4054651081081644, 'learning': 1.4054651081081644}


In [27]:
# compute_query_tfidf function

# This function determines the importance of each word in the search query "life learning" in relation to all documents.
# It splits the query into individual words and calculates the TF-IDF score for each one.
# It does this by multiplying the normalized term frequency (from query_norm_tf) by the inverse document frequency (from idf_dict_qry) for each word.
# The result is a dictionary that pairs each word with its TF-IDF score.

In [28]:
#tf-idf score for the query string("life learning")
def compute_query_tfidf(query):
    tfidf_dict_qry = {}
    sentence = query.split()
    for word in sentence:
        tfidf_dict_qry[word] = query_norm_tf[word] * idf_dict_qry[word]
    return tfidf_dict_qry
tfidf_dict_qry = compute_query_tfidf(query)
print(tfidf_dict_qry)

{'life': 0.7027325540540822, 'learning': 0.7027325540540822}


Let us now calculate the cosine similarity of the query and Document1.

Cosine Similarity(Query,Document1) = Dot product(Query, Document1) / ||Query|| * ||Document1||

Dot product(Query, Document1) 
     = ((0.702753576) * (0.140550715) + (0.702753576)*(0.140550715))
     = 0.197545035151

||Query|| = sqrt((0.702753576)2 + (0.702753576)2) = 0.993843638185

||Document1|| = sqrt((0.140550715)2 + (0.140550715)2) = 0.198768727354

Cosine Similarity(Query, Document) = 0.197545035151 / (0.993843638185) * (0.198768727354)
                                        = 0.197545035151 / 0.197545035151
                                        = 1

In [29]:
# cosine_similarity function

# This function measures how similar a search query is to a document.
# It takes the TF-IDF scores for the query and the document and caclucates the dot product.
# Then it calculates the magnitude (or length) of both the query and document vectors.
# It divides the dot product by the product of the two magnitudes to get the cosine similarity.
# The closer this number is to 1, the more similar the document is to the search query.

# There's also a helper function called 'flatten' 
# which is used to turn nested lists into a single list. 
# This can be handy when dealing with lists of lists.

In [30]:
#Cosine Similarity(Query,Document1) = Dot product(Query, Document1) / ||Query|| * ||Document1||

"""
Example : Dot roduct(Query, Document1) 

     life:
     = tfidf(life w.r.t query) * tfidf(life w.r.t Document1) +  / 
     sqrt(tfidf(life w.r.t query)) * 
     sqrt(tfidf(life w.r.t doc1))
     
     learning:
     =tfidf(learning w.r.t query) * tfidf(learning w.r.t Document1)/
     sqrt(tfidf(learning w.r.t query)) * 
     sqrt(tfidf(learning w.r.t doc1))

"""
def cosine_similarity(tfidf_dict_qry, df , query , doc_num):
    dot_product = 0
    qry_mod = 0
    doc_mod = 0
    tokens = query.split()
   
    for keyword in tokens:
        dot_product += tfidf_dict_qry[keyword] * df[keyword][df['doc'] == doc_num]
        #||Query||
        qry_mod += tfidf_dict_qry[keyword] * tfidf_dict_qry[keyword]
        #||Document||
        doc_mod += df[keyword][df['doc'] == doc_num] * df[keyword][df['doc'] == doc_num]
    qry_mod = np.sqrt(qry_mod)
    doc_mod = np.sqrt(doc_mod)
    #implement formula
    denominator = qry_mod * doc_mod
    cos_sim = dot_product/denominator
     
    return cos_sim

from collections.abc import Iterable
def flatten(lis):
     for item in lis:
        if isinstance(item, Iterable) and not isinstance(item, str):
             for x in flatten(item):
                yield x
        else:        
             yield item


In [31]:
# rank_similarity_docs function

# This function ranks documents by how similar they are to the search query.
# It goes through each document and uses the cosine_similarity function to calculate similarity.
# Each similarity score is added to a list.
# The function returns this list of cosine similarity scores, one for each document.
# It also creates a list of document names to make it clear which score belongs to which document.

In [32]:
def rank_similarity_docs(data):
    cos_sim =[]
    for doc_num in range(0 , len(data)):
        cos_sim.append(cosine_similarity(tfidf_dict_qry, df , query , doc_num).tolist())
    return cos_sim
similarity_docs = rank_similarity_docs(documents)
doc_names = [f"Document{i+1}" for i in range(len(documents))]
print(doc_names)
print(list(flatten(similarity_docs)))

['Document1', 'Document2', 'Document3']
[1.0, 0.7071067811865475, 0.7071067811865475]


In [33]:
# Document1 has a cosine similarity score of 1.0, which indicates that 
# it's perfectly aligned with the search query—it contains the terms in the query, 
# making it the most relevant document.

# Document2 and Document3 both have cosine similarity scores of approximately 0.7071, 
# suggesting that they are somewhat relevant to the search query but less so than Document1.


In [34]:
similarity_docs

[[1.0], [0.7071067811865475], [0.7071067811865475]]